# Convolutional Neural Networks
- - -

Vamos a trabajar con un conjunto de datos de imágenes de artículos de ropa.

Consiste en un conjunto de 70.000 ejemplos. 

Cada ejemplo es una imagen en escala de grises de 28x28, asociada a una target de 10 clases.

<table>
  <tr><td>
    <img src="https://1.bp.blogspot.com/-AIPR5UuydTY/WbCLlGEmoAI/AAAAAAAAA2U/Teu6q2FF9LslUL6t6Qn5YjzGWNfHC7y7wCLcBGAs/w1200-h630-p-k-no-nu/fashion-mnist-sprite.png"  width="750">
  </td></tr>
  <tr><td align="center">
    </td></tr>
</table>

Importaremos nuestras librerías:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import seaborn as sns

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

- - -

Importaremos el dataset

In [ ]:
fashion = pd.read_csv('https://raw.githubusercontent.com/4data-lab/datasets/master/fashion.zip')

In [ ]:
print('Número de filas: {}'.format(fashion.shape[0]))
print('Número de columnas: {}'.format(fashion.shape[1]))

In [ ]:
fashion.info()

In [ ]:
fashion.head()

In [ ]:
#Miramos si las clases están balanceadas
sns.countplot(x="class", data=fashion, palette="Set3")

In [ ]:
X = fashion.drop('class', 1)

In [ ]:
y = fashion['class']

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=10000, random_state=1)

- train_images: información de las imágenes de entrenamiento
- train_labels: clase del target de entrenamiento


- test_images: informacion de imágenes de test
- test_labels: clase del target de test

In [ ]:
train_images.shape

Como podemos ver en este caso, cuando estamos interpretando el shape de train_images, podemos ver que se trata de un conjunto de 60000 imágenes de 28 x 28 (784), que es la cantidad de pixeles.

In [ ]:
train_labels.shape

Por otra parte, si miramos el shape de train_labels, podemos ver que corresponde al target de entrenamiento de train_images

In [ ]:
print(train_labels[:15])

Si miramos el target, podemos ver que se trata de números enteros.

- - -
Ahora veamos cómo se ve una de las imágenes de nuestro conjunto de datos

In [ ]:
plt.figure()
plt.imshow(np.reshape(train_images.values[1], (28,28)), cmap='gray');

Los valores de nuestra imagen estan compendidos entre 0 y 255, ya que se trata de una imagen en escala de grises.

In [ ]:
plt.figure()
plt.imshow(np.reshape(train_images.values[3], (28,28)), cmap='gray')
plt.colorbar();

En el target tenemos valores que van de 0 a 9 y representan a las clases, en este caso a los tipos de ropa:

- 0	T-shirt/top

- 1	Trouser

- 2	Pullover

- 3	Dress

- 4	Coat

- 5	Sandal

- 6	Shirt

- 7	Sneaker

- 8	Bag

- 9	Ankle boot

- - -

Con un código un poco más complejo, podemos generar un subplot. Es decir, uno o más plots dentro otro.

Pasaremos las primeras 20 imágenes y a su vez las etiquetaremos.

In [ ]:
etiquetas = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
plt.figure(figsize=(10,10))
for imagenes in range(20):
    plt.subplot(5,4,imagenes+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.reshape(train_images.values[imagenes], (28,28)), cmap='gray')
    plt.xlabel(etiquetas[train_labels.values[imagenes]])

5  sera el número de filas

4 sera el número de columnas

imagenes + 1 sera el número de subdivisiones activas del subplot

- - - 
A continuación, prepararemos nuestros datos de entrada.

Debemos modificar la estructura de nuestros datos (**train_images** y **test_images**) a la forma que nuestro modelo espera.

El primer número es el número de imágenes (60000 para train_images y 10000 para test_images).
Luego viene la forma de cada imagen (28x28). El último número es 1, que significa que las imágenes están en escala de grises. Si fueran imágenes a color, tendríamos tres matrices de 28x28.

In [ ]:
train_images = train_images.values.reshape(60000,28,28,1)

In [ ]:
train_images.shape

In [ ]:
test_images = test_images.values.reshape(10000,28,28,1)

In [ ]:
test_images.shape


Escalaremos nuestros datos entre 0 y 1. Para ello simplemente podemos realizar una división por 255.

In [ ]:
train_images = train_images / 255

In [ ]:
test_images = test_images / 255

- - -
- - -
El tipo de modelo que usaremos es secuencial. Que permite construir un modelo capa por capa.

Usamos la función add() para agregar capas a nuestro modelo.

Las capas Conv2D son capas de convolución que tratarán con nuestras imágenes de entrada y que se ven como matrices bidimensionales.

El 64 en la primera capa y 32 en la segunda capa son el número de neuronas en cada capa. 
Este número se puede ajustar para que sea mayor o menor.

El tamaño del kernel es el tamaño de la matriz de filtro para nuestra convolución. 
Un kernel de 3 significa que tendremos una matriz de filtro de 3x3.

Nuestra primera capa también toma una forma de entrada, esta es la forma de cada imagen de entrada, (28,28,1)

Entre las capas Conv2D y la capa dense, hay una capa Flatten. Esta capa sirve como una conexión entre la convolución y las capas densas. Lo que está haciendo es pasar de un vector 2D a un vector de una única dimensión (aplana la matriz).

Dense es el tipo de capa que usaremos para nuestra capa de salida, es un tipo de capa estándar que se usa en muchos casos para redes neuronales.

Tendremos 10 neuronas en nuestra capa de salida, uno para cada resultado posible (0–9).

La activación función de activación de la última capa es softmax, que hace que la suma de salida sea de hasta 1 para que la salida se pueda interpretar como la probabilidad de que la entrada pertenezca a una clase concreta.
Así pues, cada neurona contiene una puntuación que indica la probabilidad de que la imagen actual pertenezca a una de las 10 clases.
El modelo luego hará su predicción en función de qué clase tiene la mayor probabilidad.


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

- - -
- - -

Antes de que el modelo esté listo para que lo entrenemos, necesitara algunos ajustes. 


- Optimizador: Usaremos 'adam' que actualmente es el optimizador de referencia. Adam (Adaptive Moment Estimation) ajusta la tasa de aprendizaje durante el entrenamiento.
- Función de coste o de error: Usaremos sparse_categorical_crossentropy, que es la opción más común para la clasificación. Una puntuación más baja indica que el modelo está funcionando mejor.

  - Si tu target son dummies, utiliza categorical_crossentropy.

    [1,0,0]

    [0,1,0]

    [0,0,1]

  - Si tu target son números enteros, utiliza sparse_categorical_crossentropy.
  
    1
  
    2
  
    3

-  Métricas: utilizaremos la accuracy para interpretar fácilmente los resultados.

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
keras.utils.plot_model(model, "NuestraCNN.png", show_shapes=True)

Ahora empezaremos a entrenar a nuestro modelo realizandole un Fit, para que aprenda de los datos de entrenamiento.

El número de épocas es el número de veces que el modelo recorrerá los datos.

Cuantas más épocas ejecutemos, más mejorará el modelo, hasta cierto punto. 

In [ ]:
history = model.fit(train_images, train_labels, validation_split = 0.1, batch_size = 256, epochs=10)

In [ ]:
accuracy = model.evaluate(train_images, train_labels)

print(accuracy[1], "Accuracy en Train",)

Ahora testearemos nuestro modelo con datos de test, para ver como lo hace con datos que no ha visto.

In [ ]:
accuracy = model.evaluate(test_images, test_labels)

print(accuracy[1], "Accuracy en Test",)

Probaremos alguna predicciones con nuestro modelo

In [ ]:
predictions = model.predict(test_images)

In [ ]:
predictions[2]

El anterior array describe la confianza del modelo al clasificar una clase.

Realizando un argmax, podemos obtener el valor de la predicción.

In [ ]:
print(np.argmax(predictions[2]), "Valor de la predicción")

In [ ]:
print(test_labels.values[2], "Valor real esperado")

- - -

In [ ]:
def plot_results(x):
    print(np.argmax(predictions[x]), "Valor de la predicción")
    print(test_labels.values[x], "Valor real esperado")
    plt.bar(range(10), predictions[x])
    plt.xticks(range(10))
    plt.ylim([0, 1])
    plt.grid(False)

In [ ]:
plot_results(21)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
label_predictions = np.argmax(predictions,axis = 1)
confusion_matrix(test_labels, label_predictions)


In [ ]:
print(classification_report(test_labels, label_predictions))

Evolución de la accuracy y la loss por cada epoch:

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

También podemos guardar el modelo

In [ ]:
# Guardar el Modelo
model.save('my_CNN.h5')


In [ ]:
# Abre el modelo entrenado desde un archivo
new_model = keras.models.load_model('my_CNN.h5')

In [ ]:
accuracy = new_model.evaluate(test_images, test_labels)

print(accuracy[1], "Accuracy en Test en el modelo guardado",)